In [1]:
from deployment_model.seq_model import SeqModel

import torch.nn as nn
import torch
import torchtext
from torchtext.data import Field
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud, STOPWORDS
import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jeremy.zhang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jeremy.zhang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
TEXT = Field(sequential=True, tokenize=word_tokenize, lower=True, stop_words=STOPWORDS)

In [3]:
LABELS = ['neu', 'neg', 'pos']

In [4]:
vocab = {}
with open("vocab.pkl", "rb") as f:
    vocab = pickle.load(f)

In [5]:
best_config = {
    "hidden_size": 302,
    "lr": 0.00010769630091763721,
    "l2": 2.5888680371842294e-05,
    "nonlin": "tanh",
    "dropout": 0.1,
    "num_layers": 2,
    "mode": 0,
    "optimizer": "Adam",
    "momentum": 0.1,
}

best_model = SeqModel(
    embedding_size=100,
    vocab_size=len(vocab),
    output_size=3,
    hidden_size=best_config["hidden_size"],
    num_layers=best_config["num_layers"],
    nonlin=best_config["nonlin"],
    dropout_rate=best_config["dropout"],
    mode=best_config["mode"],
    unit="gru",
    more_features=False,
)

In [6]:
best_model.load_state_dict(torch.load("model_deploy.pt"))

<All keys matched successfully>

In [7]:
new_input = "vaccine will work eventually, I am very happy and my family"
print(TEXT.preprocess(new_input))
trans_input = [vocab.get(token, 0) for token in TEXT.preprocess(new_input)]
trans_input

['vaccine', 'will', 'work', 'eventually', ',', 'happy', 'family']


[2, 7, 30, 2000, 0, 498, 336]

In [8]:
model_outputs = best_model(torch.LongTensor(trans_input).reshape((-1,1)))
model_outputs

tensor([[-1.3882, -2.4655, -0.4072]], grad_fn=<LogSoftmaxBackward>)

In [9]:
probabilities, predicted = torch.max(model_outputs.cpu().data, 1)

In [11]:
pred_label = LABELS[predicted]
probabilities

tensor([-0.4072])